## Lesson-01 Assignment

>

## 本次作业的内容

>

## 基础理论部分

#### 0. Can you come up out 3 sceneraies which use AI methods? 

Ans: 
1）文本分类
2）机器翻译
3）文本生成

#### 1. How do we use Github; Why do we use Jupyter and Pycharm;

Ans: 
1）Jupyter对交互性较好，可以一条一条代码运行出结果。
2）Pycharm兼容性较好，适用于项目开发。

#### 2. What's the Probability Model?

Ans:
概率模型是描述不同随机变量之间关系的数学模型，通常情况下刻画了一个或多个随机变量之间的相互非确定性的概率关系。

#### 3. Can you came up with some sceneraies at which we could use Probability Model?

Ans:
1）朴素贝叶斯
2）语言模型
3）高斯分布

#### 4. Why do we use probability and what's the difficult points for programming based on parsing and pattern match?

Ans:
概率能帮助我们弄清不确定时间发生的几率是多少。
基于解析和模式匹配的编程的难点在于难以构建一个通用泛化的模型函数？

#### 5. What's the Language Model;

Ans:
语言模型是指一句话中的每个字或词组成一个句子的概率。

#### 6. Can you came up with some sceneraies at which we could use Language Model?


Ans:
1）语音识别
2）词性标注
3）命名实体识别

#### 7. What's the 1-gram language model;

Ans:
每个字或词出现的概率，不与上下文有关，只与本身相关。

#### 8. What's the disadvantages and advantages of 1-gram language model;

Ans:
劣势：
1）无法识别语序颠倒的情况，即无语法结构。
2）字或词之间没有关联，也没有约束。

优势：
1）概率独立分布，计算复杂度较低。
2）词典的维度较小。


#### 9. What't the 2-gram models;

Ans:
每个字或词与它最近的一个字或词相关联，受到最近一个字或词的约束。

## 编程实践部分

#### 1. 设计你自己的句子生成器

定义自己的语法: 

第一个语法：

In [129]:
grammar_1 = '''
语法 => 主语 谓语 宾语
主语 => 你 | 我 | 他 | 她 | 它
谓语 => 爱 | 喜欢 | 想 | 怀念
宾语 => 你 | 我 | 他 | 她 | 它
'''

第二个语法：

In [130]:
grammar_2 = '''
语法 => 时间 人物 助动词 地点 动作
时间 => 昨天 时辰 | 今天 时辰 | 明天 时辰
时辰 => null | 早上 | 中午 | 下午 | 晚上
人物 => 我 | 你 | 他
助动词 => 坐船去 | 坐火箭飞去 | 开车到 | 走路去
地点 => 游乐场 | 学校 | 停车场 | 实验室 | 小岛 | 外太空
动作 => 玩耍 | 学习 | 停车 | 做实验
'''

In [131]:
def create_grammar(grammar_str, split='=>', line_split='\n'):
    grammar = {}
    for line in grammar_str.split(line_split):
        if not line.strip():  # 去掉line前后空格后，line为空时，continue。
            continue
        exp, stmt = line.split(split)
        grammar[exp.strip()] = [s.split() for s in stmt.split('|')]
    return grammar

In [132]:
grammar_1 = create_grammar(grammar_1)
print(grammar_1)

{'语法': [['主语', '谓语', '宾语']], '主语': [['你'], ['我'], ['他'], ['她'], ['它']], '谓语': [['爱'], ['喜欢'], ['想'], ['怀念']], '宾语': [['你'], ['我'], ['他'], ['她'], ['它']]}


In [126]:
grammar_2 = create_grammar(grammar_2)
print(grammar_2)

{'语法': [['时间', '人物', '助动词', '地点', '动作']], '时间': [['昨天', '时辰'], ['今天', '时辰'], ['明天', '时辰']], '时辰': [['null'], ['早上'], ['中午'], ['下午'], ['晚上']], '人物': [['我'], ['你'], ['他']], '助动词': [['坐船去'], ['坐火箭飞去'], ['开车到'], ['走路去']], '地点': [['游乐场'], ['学校'], ['停车场'], ['实验室'], ['小岛'], ['外太空']], '动作': [['玩耍'], ['学习'], ['停车'], ['做实验']]}


In [120]:
def generate(gram, target='语法'):
    if target not in gram:
        return target
    import random
    expaned = [generate(gram, t) for t in random.choice(gram[target])]
    
    return ''.join([e if e != '/n' else '\n' for e in expaned if e != 'null'])


In [121]:
g = generate(gram=grammar, target='语法')
print(g)

今天早上我开车到实验室学习


In [98]:
def generate_n(n, gram=grammar, target='语法'):
    for _ in range(n):
        g = generate(gram, target)
        print(g)

generate_n(5)

明天早上我开车到外太空做实验
今天中午他坐船去学校玩耍
昨天他走路去外太空学习
明天下午我走路去实验室做实验
今天下午他开车到游乐场停车


>

#### 2. 使用新数据源完成语言模型的训练

按照我们上文中定义的`prob_2`函数，我们更换一个文本数据源，获得新的Language Model:

1. 下载文本数据集（你可以在以下数据集中任选一个，也可以两个都使用）
    + 可选数据集1，保险行业问询对话集： https://github.com/Computing-Intelligence/insuranceqa-corpus-zh/raw/release/corpus/pool/train.txt.gz
    + 可选数据集2：豆瓣评论数据集：https://github.com/Computing-Intelligence/datasource/raw/master/movie_comments.csv
2. 修改代码，获得新的**2-gram**语言模型
    + 进行文本清洗，获得所有的纯文本
    + 将这些文本进行切词
    + 送入之前定义的语言模型中，判断文本的合理程度

In [99]:
import pandas as pd
import random
from collections import Counter
import re
import jieba

In [100]:
def token(string):
    # we will learn the regular expression next course.
    return re.findall('\w+', string)

def cut(string):
    return list(jieba.cut(string))

In [28]:
filename = 'movie_comments.csv'

content = pd.read_csv(filename, encoding='utf-8')
content['comment'].head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0                                   吴京意淫到了脑残的地步，看了恶心想吐
1    首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...
2    吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...
3                        凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。
4                                                 中二得很
Name: comment, dtype: object

In [29]:
articles = content['comment'].tolist()
len(articles)

261497

In [30]:
articles_clean = [''.join(token(str(a)))for a in articles]

with open('articles_clean.txt', 'w') as f:
    for i, a in enumerate(articles_clean):
        if i % 50000 == 0:
            print(i)
        f.write(a + '\n')

0
50000
100000
150000
200000
250000


In [31]:
TOKEN = []
for i, line in enumerate(open('articles_clean.txt')):
    if i % 10000 == 0:
        print(i)
    TOKEN += cut(line)

Building prefix dict from the default dictionary ...


0


Dumping model to file cache /var/folders/55/dcp8q8wx08ld4bfxx_9726d40000gn/T/jieba.cache
Loading model cost 0.799 seconds.
Prefix dict has been built succesfully.


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000


In [33]:
words_count = Counter(TOKEN)
len(words_count)

164393

In [34]:
words_count.most_common(100)

[('的', 328262),
 ('\n', 261497),
 ('了', 102420),
 ('是', 73106),
 ('我', 50338),
 ('都', 36255),
 ('很', 34712),
 ('看', 34022),
 ('电影', 33675),
 ('也', 32065),
 ('和', 31290),
 ('在', 31245),
 ('不', 28435),
 ('有', 27939),
 ('就', 25685),
 ('人', 23909),
 ('好', 22858),
 ('啊', 20803),
 ('这', 17484),
 ('还', 17449),
 ('一个', 17343),
 ('你', 17282),
 ('还是', 16425),
 ('但', 15578),
 ('故事', 15010),
 ('没有', 14343),
 ('就是', 14007),
 ('喜欢', 13566),
 ('让', 13304),
 ('太', 12676),
 ('又', 11566),
 ('剧情', 11359),
 ('没', 10858),
 ('说', 10764),
 ('吧', 10747),
 ('他', 10675),
 ('不错', 10416),
 ('得', 10349),
 ('到', 10341),
 ('给', 10300),
 ('这个', 10058),
 ('上', 10054),
 ('被', 9939),
 ('对', 9824),
 ('最后', 9694),
 ('一部', 9693),
 ('片子', 9590),
 ('什么', 9571),
 ('能', 9532),
 ('与', 9168),
 ('多', 8977),
 ('可以', 8972),
 ('不是', 8811),
 ('最', 8669),
 ('觉得', 8626),
 ('中', 8446),
 ('导演', 8390),
 ('自己', 8354),
 ('拍', 8172),
 ('好看', 8085),
 ('要', 8081),
 ('真的', 7908),
 ('感觉', 7828),
 ('但是', 7723),
 ('里', 7655),
 ('那', 7503),
 ('有点',

In [133]:
def prob_1(word):
    return words_count[word] / len(TOKEN)

def prob_2(word1, word2):
    if word1 + word2 in words_count_2:
        return words_count_2[word1+word2] / words_count[word1]
    else:
        return 1 / len(TOKEN_2_GRAM)

In [134]:
TOKEN_2_GRAM = [''.join(TOKEN[i:i+2]) for i, _ in enumerate(TOKEN[:-2])]

In [135]:
words_count_2 = Counter(TOKEN_2_GRAM)

In [136]:
def get_probablity(sentence):
    words = cut(sentence)
    sentence_pro = 1
    
    for i, word in enumerate(words[:-1]):
        next_ = words[i+1]
        probability = prob_2(word, next_)
        sentence_pro *= probability
    
    return sentence_pro

In [137]:
get_probablity('首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。')

1.8600395709893428e-54

#### 3. 获得最优质的的语言

当我们能够生成随机的语言并且能判断之后，我们就可以生成更加合理的语言了。请定义 generate_best 函数，该函数输入一个语法 + 语言模型，能够生成**n**个句子，并能选择一个最合理的句子: 



In [138]:
def token_from_txt(txt):
    TOKEN = []
    for i, line in enumerate(open(txt)):
        TOKEN += cut(line)
    return TOKEN

def prob_2(word1, word2):
    if word1 + word2 in words_count_2:
        return words_count_2[word1+word2] / len(TOKEN_2_GRAM)
    else:
        return 1 / len(TOKEN_2_GRAM)

def get_probablity(sentence):
    words = cut(sentence)
    sentence_pro = 1
    
    for i, word in enumerate(words[:-1]):
        next_ = words[i+1]
        probability = prob_2(word, next_)
        sentence_pro *= probability
    return sentence_pro

def create_grammar(grammar_str, split='=>', line_split='\n'):
    grammar = {}
    for line in grammar_str.split(line_split):
        if not line.strip():  # 去掉line前后空格后，line为空时，continue。
            continue
        exp, stmt = line.split(split)
        grammar[exp.strip()] = [s.split() for s in stmt.split('|')]
    return grammar

def generate(gram, target='语法'):
    if target not in gram:
        return target
    import random
    expaned = [generate(gram, t) for t in random.choice(gram[target])]
    
    return ''.join([e if e != '/n' else '\n' for e in expaned if e != 'null'])

def generate_best(gram, n):
    ls = []
    for _ in range(n):
        temp_s = generate(gram)
        temp_p = get_probablity(temp_s)
        ls.append((temp_s, temp_p))
    for i, j in ls:
        print(i, '此句子合理的概率为：', j)
    print('\n其中最合理的句子为：', sorted(ls, key=lambda x: x[1], reverse=True)[0])

In [139]:
TOKEN = token_from_txt('grammar_2.txt')
TOKEN_2_GRAM = [''.join(TOKEN[i:i+2]) for i, _ in enumerate(TOKEN[:-2])]
words_count_2 = Counter(TOKEN_2_GRAM)

grammar_2 = '''
语法 => 时间 人物 助动词 地点 动作
时间 => 昨天 时辰 | 今天 时辰 | 明天 时辰
时辰 => null | 早上 | 中午 | 下午 | 晚上
人物 => 我 | 你 | 他
助动词 => 坐船去 | 坐火箭飞去 | 开车到 | 走路去
地点 => 游乐场 | 学校 | 停车场 | 实验室 | 小岛 | 外太空
动作 => 玩耍 | 学习 | 停车 | 做实验
'''
grammar = create_grammar(grammar_2)

generate_best(gram=grammar, n=10)

昨天早上我走路去外太空玩耍 此句子合理的概率为： 3.588677504933481e-10
今天你坐火箭飞去停车场做实验 此句子合理的概率为： 2.436856612675519e-14
昨天中午你开车到小岛学习 此句子合理的概率为： 4.8447146316602e-10
昨天下午他走路去学校玩耍 此句子合理的概率为： 5.383016257400222e-10
今天晚上他坐火箭飞去学校玩耍 此句子合理的概率为： 8.122855375585064e-15
明天晚上他开车到游乐场做实验 此句子合理的概率为： 1.4621139676053112e-13
今天早上他开车到小岛停车 此句子合理的概率为： 2.290645215914988e-12
昨天中午他开车到小岛停车 此句子合理的概率为： 3.229809754440133e-10
今天下午我开车到游乐场玩耍 此句子合理的概率为： 5.81365755799224e-09
明天下午你坐火箭飞去学校停车 此句子合理的概率为： 8.122855375585064e-15

其中最合理的句子为： ('今天下午我开车到游乐场玩耍', 5.81365755799224e-09)


好了，现在我们实现了自己的第一个AI模型，这个模型能够生成比较接近于人类的语言。

>

Q: 这个模型有什么问题？ 你准备如何提升？ 

Ans:
1）需要大规模的语料库才能确保准确率的提升，在grammar_2.txt中多加些正确的例子。
2）句子长度越长，句子合理的概率越小？
3）可采用3-Gram进行尝试。